In [1]:
import sys
sys.path.append('/home/ian/code/github')

In [2]:
from utils.pymongo_wrapper import pymongo_wrapper
pmw = pymongo_wrapper()
table = pmw.get_collection('jiankang39','diseases')

In [10]:
c=list(pmw.find_all(table,fieldlist=['疾病名称','简介','典型症状','多发人群']))

In [4]:
from utils.pandas_wrapper import pandas_wrapper as pdw

In [12]:
l1 = list()
l2 = list()
l3 = list()
for i in c:
    if '多发人群' in i:
        l1.append(i['多发人群'])
    else:
        l1.append(None)
    if '简介' in i:
        l2.append(i['简介'])
    else:
        l2.append(None)
    if '典型症状' in i:
        l3.append(','.join(i['典型症状']))
    else:
        l3.append(None)

In [13]:
df = pdw.build_df_from_dict({'疾病名称':[i['疾病名称'] for i in c],'简介':l2,'典型症状':l3,'多发人群':l1})
df.head()

,疾病名称,简介,典型症状,多发人群
0,偏头痛,偏头痛是一类有家族发病倾向的周期性发作疾病。表现为发作性的偏侧搏动性头痛，伴恶心、呕吐及羞明...,"单侧头痛,电击样痛或刺痛的头痛,头痛为搏动性",女性占2/3以上， 10岁前、20岁前、40岁前发病分别为25%， 55%， 90%。
1,帕金森,帕金森病(Parkinsons disease)又称震颤麻痹，巴金森氏症或柏金逊症，多在60...,"静止性震颤,肌张力过高,感觉障碍",老年人，于40～70岁，60岁后发病率增高，30多岁前发病少见。
2,老年痴呆症,阿尔茨海默病(Alzheimer’s disease，AD)是慢性进行性中枢神经系统变性病导...,None,None
3,蛛网膜下腔出血,蛛网膜下腔出血(subarachnoid hemorrhage，SAH)是出血性脑血管病的一...,"脑膜刺激症状,定位神经体征,偏瘫",30～40岁青壮年多见，也有的报道80%的发病年龄在30～69岁。男性稍多于女性。
4,小儿多动症,儿童多动症、多动综合征(hyperkinetic syndrome of childhood...,"注意力不集中,情绪起伏大,学习困难",婴幼儿


In [15]:
df.to_csv('/home/ian/code/data/daozhen/39疾病列表.csv',encoding='utf8')